In [ ]:
# default_exp core

# Voynich decoding attempts

> If this is a Vigenere cipher, the Kasiski test could decrypt.  (Look for repeated groups of words, when a repeat word is ciphered the same way)

See also Frontenac, Nostrad

In [ ]:
#hide
from nbdev.showdoc import *

Kasiki test
# from Voynich information browser
http://voynich.freie-literatur.de/index.php?show=extractor


In [ ]:
# frequency count using  code from https://gist.github.com/nmpowell/f3faf7496bf9d915579f 'Nick Powell (PhD student, CMIC & CABI, UCL, UK), nicholas.powell.11@ucl.ac.uk'
import os
import sys
import string
import argparse
import operator

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from collections import Counter

In [ ]:
#put file info here
rawfilepath = 'transcriptions/f1r_autocap.txt'

#and number of results desired
top_n = 200

In [ ]:
# Load the file
    filepath = os.path.normpath(os.path.join(rawfilepath))
    file = open(filepath, 'r')

In [ ]:
 # Parse as a list, removing lines
    content_sublists = [line.split(',') for line in file.readlines()]

In [ ]:
 # Parse into a single list (from a list of lists)
    content_list = [item for sublist in content_sublists for item in sublist]

In [ ]:
 # Remove whitespace so we can concatenate appropriately, and unify case
    content_list_strip = [str.strip().lower() for str in content_list]

In [ ]:
 # Concatenate strings into a single string
    content_concat = ' '.join(content_list_strip)

In [ ]:
#  MP- TESTING this step to replace period with space?
 #  content_concat2 = [w.replace('.', ' ') for w in content_concat]
content_concat2 = content_concat.replace(".", " ") 


In [ ]:
# Remove punctuation and new lines #don't remove punctuation, it's important in the eva transcription style
    punct = set(string.punctuation)
    unpunct_content = ''.join(x for x in content_concat2)
                              
                              #if x not in punct)

In [ ]:
# Split string into list of strings, again
    word_list = unpunct_content.split()

In [ ]:
 # Perform count
    counts_all = Counter(word_list)
    words, count_values = zip(*counts_all.items())

In [ ]:
# Sort both lists by frequency in values (Schwartzian transform) - thanks, http://stackoverflow.com/questions/9543211/sorting-a-list-in-python-using-the-result-from-sorting-another-list
    values_sorted, words_sorted = zip(*sorted(zip(count_values, words), key=operator.itemgetter(0), reverse=True))

In [ ]:
 # Top N
    words_sorted_top = words_sorted[0:top_n]
    values_sorted_top = values_sorted[0:top_n]

In [ ]:
print("- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -")
print("{0} unique words identified in the text file, {1}".format(len(values_sorted), filepath))
print("The top {0} words are: \n{1}".format(top_n, words_sorted_top))
print("... their respective frequencies: \n{0}".format(values_sorted_top))
print("- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -")

# Pandas DataFrame just for visualisation
df = pd.DataFrame({'count': values_sorted_top, 'word': words_sorted_top})
print("{0}".format(df))
sys.stdout.flush()

In [ ]:
 # Histogram
    
# Make xticklabels comprehensible by matplotlib
xticklabels = str(list(words_sorted_top)).split()

# Remove the single quotes, commas and enclosing square brackets
xtlabs = [xstr.replace("'","").replace(",","").replace("]","").replace("[","") for xstr in xticklabels]

In [ ]:
indices = np.arange(len(words_sorted_top))
width = 1
fig = plt.figure()
fig.suptitle('Word frequency histogram, top {0}'.format(top_n), fontsize=16)
plt.xlabel('word', fontsize=12)
plt.ylabel('count', fontsize=12)
plt.bar(indices, values_sorted_top, width)
plt.xticks(indices + width * 0.5, xtlabs, rotation='vertical', fontsize=8)
plt.show()


In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
df


Sort alphabetically

In [ ]:
# ascending
df.sort_values('word')

In [ ]:
df['word']

Idea: take off the first "letter" of each word

In [ ]:
word_list

In [ ]:
for word in word_list:
    print(word[0], sep='', end='')

In [ ]:
%%html
<div class=eva>
fyaasscyksscoykcsaccdssoysccdsooorcdooodyccccy*oo!scosscsysoocodckdscsdot*ycoscyscokooossdsckcccsoddococcodckdsccskcckcscskkdcskytcspdcdysskcddcdsocr!cdsccdcodcssctspcrd*cdcdskdsc!ksodfcdcyoodok*kddskcokkdc!ccccd
<div/>

In [ ]:
from nbdev.export import *
notebook2script()

In [ ]:
# CUSTOM CSS LOAD
from IPython.core.display import HTML
def css_styling():
    styles = open("styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

In [ ]:
%%html
<div class=eva>
This is a test
<div/>

In [ ]:
HTML('<div class=eva>{{ unpunct_content }}<div/>')

In [ ]:
words_sorted_top

In [ ]:
%%html
<div class=eva>

 
 chol  <br>
 daiin <br>
 dain  <br>
 shol  <br>
 y     <br>
 chor  <br>
 cthy  <br>
 shey  <br>
 kor   <br>
 cthar
<div/>

simple cipher substitution attempt


using code from: http://practicalcryptography.com/media/cryptanalysis/files/break_simplesub_2.py

In [3]:
from pycipher import SimpleSubstitution as SimpleSub
import random
import re
#from ngram_score import ngram_score
#fitness = ngram_score('english_quadgrams.txt') # load our quadgram statistics


In [15]:
#ctext='pmpafxaikkitprdsikcplifhwceigixkirradfeirdgkipgigudkcekiigpwrpucikceiginasikwduearrxiiqepcceindgmieinpwdfprduppcedoikiqiasafmfddfipfgmdafmfdteiki'
ctext='fyaasscyksscoykcsaccdssoysccdsooorcdooodyccccy*oo!scosscsysoocodckdscsdot*ycoscyscokooossdsckcccsoddococcodckdsccskcckcscskkdcskytcspdcdysskcddcdsocr!cdsccdcodcssctspcrd*cdcdskdsc!ksodfcdcyoodok*kddskcokkdc!ccccd'
ctext = re.sub('[^A-Z]','',ctext.upper())


In [16]:
maxkey = list('ABCDEFGHIJKLMNOPQRSTUVWXYZ')
maxscore = -99e9
parentscore,parentkey = maxscore,maxkey[:]
print("Substitution Cipher solver, you may have to wait several iterations")
print("for the correct result. Press ctrl+c to exit program.")
# keep going until we are killed by the user
i = 0
while 1:
    i = i+1
    random.shuffle(parentkey)
    deciphered = SimpleSub(parentkey).decipher(ctext)
    #parentscore = fitness.score(deciphered)
    count = 0
    while count < 1:
        a = random.randint(0,25)
        b = random.randint(0,25)
        child = parentkey[:]
        # swap two characters in the child
        child[a],child[b] = child[b],child[a]
        deciphered = SimpleSub(child).decipher(ctext)
        # score = fitness.score(deciphered)
        # if the child was better, replace the parent with it
        #if score > parentscore:
         #   parentscore = score
          #  parentkey = child[:]
           # count = 0
        print('Key:')
        print(parentkey)
        print('Deciphered:')
        print(deciphered)
        print('Iteration:')
        print(i)
        count = count+1
    # keep track of best score seen so far
   # if parentscore>maxscore:
    #    maxscore,maxkey = parentscore,parentkey[:]
     #   print '\nbest score so far:',maxscore,'on iteration',i
      #  ss = SimpleSub(maxkey)
       # print '    best key: '+''.join(maxkey)
        #print '    plaintext: '+ss.decipher(ctext)


Substitution Cipher solver, you may have to wait several iterations
for the correct result. Press ctrl+c to exit program.
Key:
['E', 'H', 'B', 'M', 'A', 'O', 'J', 'Q', 'U', 'K', 'P', 'T', 'N', 'F', 'S', 'I', 'D', 'X', 'R', 'L', 'V', 'Z', 'W', 'G', 'Y', 'C']
Deciphered:
NYCCOOZYJOOZFYJZOCZZQOOFYOZZQOFFFSZQFFFQYZZZZYFFOZFOOZOYOFFZFQZJQOZOQFLYZFOZYOZFJFFFOOQOZJZZZOFQQFZFZZFQZJQOZZOJZZJZOZOJJQZOJYLZOKQZQYOOJZQQZQOFZSZQOZZQZFQZOOZLOKZSQZQZQOJQOZJOFQNZQZYFFQFJJQQOJZFJJQZZZZZQ
Iteration:
1
Key:
['R', 'D', 'N', 'K', 'A', 'T', 'Z', 'B', 'S', 'M', 'O', 'I', 'H', 'C', 'G', 'W', 'Q', 'E', 'X', 'J', 'U', 'F', 'Y', 'P', 'V', 'L']
Deciphered:
VWEEIINWDIINKWDNIENNBIIKWINNBIKKKANBKKKBWNNNNWKKINKIINIWIKKNKBNDBINIBKFWNKINWINKDKKKIIBINDNNNIKBBKNKNNKBNDBINNIDNNDNINIDDBNIDWFNIXBNBWIIDNBBNBIKNANBINNBNKBNIINFIXNABNBNBIDBINDIKBVNBNWKKBKDDBBIDNKDDBNNNNNB
Iteration:
2
Key:
['T', 'B', 'M', 'N', 'G', 'O', 'E', 'W', 'K', 'Y', 'Z', 'U', 'C', 'P', 'V', 'Q', 'R', 'X', 'J', 'F', 'L', 'H', 'I', 'A', 'D', 'S']
Deciphered

Key:
['L', 'Q', 'U', 'G', 'F', 'V', 'O', 'C', 'N', 'A', 'D', 'K', 'X', 'B', 'S', 'P', 'Y', 'M', 'E', 'T', 'R', 'I', 'H', 'Z', 'J', 'W']
Deciphered:
EQJJOOHQBOOHGQBHOJHHKOOGQOHHKOGGGUHKGGGKQHHHHQGGOHGOOHOQOGGHGKHBKOHOKGTQHGOHQOHGBGGGOOKOHBHHHOGKKGHGHHGKHBKOHHOBHHBHOHOBBKHOBQTHOPKHKQOOBHKKHKOGHUHKOHHKHGKHOOHTOPHUKHKHKOBKOHBOGKEHKHQGGKGBBKKOBHGBBKHHHHHK
Iteration:
45
Key:
['V', 'J', 'E', 'U', 'Q', 'H', 'Z', 'R', 'A', 'L', 'T', 'M', 'Y', 'W', 'S', 'D', 'X', 'I', 'C', 'P', 'K', 'B', 'G', 'N', 'F', 'O']
Deciphered:
YMIIOOSMUOOSZMUSOISSPOOZMOSSPOZZZHSPZZZPMSSSSMZZOSZOOSOMOZZSZPSUPOSOPZKMSZOSMOSZUZZZOOPOSUSSSOZPPZSZSSZPSUPOSSOUSSUSOSOUUPSOUMKSOTPSPMOOUSPPSPOZSHSPOSSPSZPSOOSKOTSHPSPSPOUPOSUOZPYSPSMZZPZUUPPOUSZUUPSSSSSP
Iteration:
46
Key:
['B', 'H', 'I', 'S', 'W', 'U', 'A', 'L', 'P', 'V', 'G', 'O', 'M', 'E', 'Z', 'J', 'X', 'R', 'C', 'F', 'T', 'N', 'Q', 'Y', 'D', 'K']
Deciphered:
TDGGXXSDZXXSLDZSXGSSYXXLDXSSYXLLLRSYLLLYDSSSSDLLXSLXXSXDXLLSLYSZYXSXYLUDSLXSDXSLZLLLXXYXSZSSSXLYYLSLSSLYSZYXSSXZSSZSXS

['S', 'D', 'X', 'A', 'O', 'T', 'R', 'W', 'Q', 'L', 'Y', 'V', 'I', 'Z', 'H', 'J', 'G', 'P', 'M', 'F', 'E', 'N', 'U', 'B', 'C', 'K']
Deciphered:
QKDDAAYKZAAYEKZYADYYBAAEKAYYBAEEEGYBEEEBKYYYYKEEAYEAAYAKAEEYEBYZBAYABEFKYEAYKAYEZEEEAABAYZYYYAEBBEYEYYEBYZBAYYAZYYZYAYAZZBYAZKFYARBYBKAAZYBBYBAEYGYBAYYBYEBYAAYFARYGBYBYBAZBAYZAEBQYBYKEEBEZZBBAZYEZZBYYYYYB
Iteration:
92
Key:
['G', 'U', 'W', 'X', 'F', 'Z', 'I', 'T', 'Y', 'K', 'H', 'D', 'B', 'A', 'C', 'V', 'J', 'O', 'P', 'R', 'L', 'M', 'N', 'Q', 'S', 'E']
Deciphered:
EIYYNNOIJNNORIJONYOOLNNRINOOLNRRRTOLRRRLIOOOOIRRNORNNONINRRORLOJLNONLRHIORNOINORJRRRNNLNOJOOONRLLROROORLOJLNOONJOOJONONJJLONJIHONSLOLINNJOLLOLNROTOLNOOLORLONNOHNSOTLOLOLNJLNOJNRLEOLOIRRLRJJLLNJORJJLOOOOOL
Iteration:
93
Key:
['L', 'Y', 'U', 'T', 'D', 'Z', 'P', 'H', 'F', 'M', 'E', 'K', 'X', 'J', 'R', 'I', 'O', 'G', 'A', 'C', 'N', 'Q', 'V', 'B', 'W', 'S']
Deciphered:
IBSSTTZBLTTZQBLZTSZZETTQBTZZETQQQOZEQQQEBZZZZBQQTZQTTZTBTQQZQEZLETZTEQDBZQTZBTZQLQQQTTETZLZZZTQEEQZQZZQEZLETZZTLZZLZTZTLLEZ

Key:
['P', 'U', 'Z', 'Q', 'K', 'L', 'V', 'I', 'W', 'E', 'D', 'H', 'F', 'M', 'S', 'O', 'T', 'N', 'J', 'A', 'R', 'X', 'C', 'G', 'B', 'Y']
Deciphered:
MZTTOOWZEOOWPZEWOTWWKOOPZOWWKOPPPUWKPPPKZWWWWZPPOWPOOWOZOPPWPKWEKOWOKPQZWPOWZOWPEPPPOOKOWEWWWOPKKPWPWWPKWEKOWWOEWWEWOWOEEKWOEZQWOAKWKZOOEWKKWKOPWUWKOWWKWPKWOOWQOAWUKWKWKOEKOWEOPKMWKWZPPKPEEKKOEWPEEKWWWWWK
Iteration:
140
Key:
['B', 'P', 'S', 'L', 'F', 'V', 'C', 'N', 'K', 'O', 'I', 'A', 'E', 'Z', 'R', 'X', 'T', 'D', 'W', 'U', 'Y', 'H', 'G', 'M', 'Q', 'J']
Deciphered:
EULLCCGUICCGKUIGCLGGRCCKUCGGRCKKKOGRKKKRUGGGGUKKCGKCCGCUCKKGKRGIRCGCRKQUGKCGUCGKIKKKCCRCGIGGGCKRRKGKGGKRGIRCGGCIGGIGCGCIIRGCIUQGCBRGRUCCIGRRGRCKGOGRCGGRGKRGCCGQCBGORGRGRCIRCGICKREGRGUKKRKIIRRCIGKIIRGGGGGR
Iteration:
141
Key:
['S', 'M', 'L', 'H', 'X', 'Q', 'I', 'Y', 'W', 'T', 'O', 'A', 'E', 'Z', 'D', 'C', 'R', 'F', 'J', 'P', 'B', 'N', 'U', 'V', 'K', 'G']
Deciphered:
KHLLAAPHYAAPRHYPALPPOAARHAPPOARRRQPORRROHPPPPHRRAPRAAPAHARRPROPYOAPAORJHPRAPHAPRYRRRAAOAPYPPPAROORPRPPROPYOAPPAYPPYP

Key:
['Y', 'X', 'A', 'Z', 'K', 'S', 'V', 'O', 'U', 'Q', 'E', 'I', 'N', 'R', 'T', 'M', 'H', 'C', 'G', 'L', 'D', 'F', 'J', 'P', 'B', 'W']
Deciphered:
VAIIFFRAEFFRHAERFIRRUFFHAFRRUFHHHNRUHHHUARRRRAHHFRHFFRFAFHHRHUREUFRFUHOARHFRAFRHEHHHFFUFRERRRFHUUHRHRRHUREUFRRFERRERFRFEEURFEAORFXURUAFFERUURUFHRNRUFRRURHURFFROFXRNURURUFEUFREFHUVRURAHHUHEEUUFERHEEURRRRRU
Iteration:
187
Key:
['P', 'D', 'G', 'L', 'T', 'I', 'S', 'V', 'O', 'E', 'N', 'A', 'Q', 'X', 'W', 'M', 'J', 'C', 'Z', 'R', 'F', 'B', 'U', 'H', 'K', 'Y']
Deciphered:
UZLLGGRZYGGRIZYRGLRRBGGIZGRRBGIIITRBIIIBZRRRRZIIGRIGGRGZGIIRIBRYBGRGBIEZRIGRZGRIYIIIGGBGRYRRRGIBBIRIRRIBRYBGRRGYRRYRGRGYYBRGYZERGABRBZGGYRBBRBGIRTRBGRRBRIBRGGREGARTBRBRBGYBGRYGIBURBRZIIBIYYBBGYRIYYBRRRRRB
Iteration:
188
Key:
['V', 'B', 'H', 'Z', 'J', 'X', 'Q', 'A', 'C', 'Y', 'R', 'P', 'U', 'L', 'T', 'K', 'O', 'S', 'E', 'I', 'N', 'F', 'W', 'G', 'D', 'M']
Deciphered:
AJHHRRIJPRRIQJPIRHIIYRRQJRIIYRQQQKIYQQQYJIIIIJQQRIQRRIRJRQQIQYIPYRIRYQOJIQRIJRIQPQQQRRYRIPIIIRQYYQIQIIQYIPYRIIRPIIPI

...

Key:
['G', 'J', 'M', 'A', 'Q', 'N', 'W', 'U', 'X', 'C', 'F', 'D', 'V', 'R', 'P', 'E', 'Y', 'S', 'K', 'I', 'H', 'Z', 'O', 'T', 'B', 'L']
Deciphered:
KQDDRRAQSRRAWQSARDAALRRWQRAALRWWWNALWWWLQAAAAQWWRAWRRARQRWWAWLASLRARLWXQAWRAQRAWSWWWRRLRASAAARWLLWAWAAWLASLRAARSAASARARSSLARSQXAROLALQRRSALLALRWANALRAALAWLARRAXROANLALALRSLRASRWLKALAQWWLWSSLLRSAWSSLAAAAAL
Iteration:
4088
Key:
['X', 'I', 'T', 'N', 'Y', 'F', 'Z', 'E', 'R', 'A', 'L', 'H', 'S', 'V', 'B', 'M', 'G', 'P', 'U', 'C', 'W', 'Q', 'O', 'K', 'J', 'D']
Deciphered:
FEJJMMTEXMMTWEXTMJTTZMMWEMTTZMWWWITZWWWZETTTTEWWMTWMMTMEMWWTWZTXZMTMZWCETWMTEMTWXWWWMMZMTXTTTMWZZWTWTTWZTXZMTTMXTTXTMTMXXZTMXECTMRZTZEMMXTZZTZMWTITZMTTZTWZTMMTCMRTIZTZTZMXZMTXMWZFTZTEWWZWXXZZMXTWXXZTTTTTZ
Iteration:
4089
Key:
['G', 'D', 'Q', 'U', 'C', 'M', 'K', 'F', 'Y', 'S', 'O', 'J', 'X', 'W', 'H', 'T', 'Z', 'A', 'P', 'N', 'E', 'V', 'L', 'R', 'B', 'I']
Deciphered:
GIRRJJEIHJJEKIHEJREEBJJKIJEEBJKKKXEBKKKBIEEEEIKKJEKJJEJIJKKEKBEHBJEJBKPIEKJEIJEKHKKKJJBJEHEEEJKBBKEKEEKBEHBJEEJHEE

Key:
['A', 'M', 'J', 'T', 'P', 'D', 'F', 'Z', 'H', 'B', 'I', 'W', 'S', 'N', 'Q', 'U', 'Y', 'L', 'O', 'G', 'E', 'K', 'C', 'R', 'X', 'V']
Deciphered:
GQAAMMWQVMMWSQVWMAWWFMMSQMWWFMSSSXWFSSSFQWWWWQSSMWSMMWMQMSSWSFWVFMWMFSDQWSMWQMWSVSSSMMFMWVWWWMSFFSWSWWSFWVFMWWMVWWVWMWMVVFWMVQDWMEFWFQMMVWFFWFMSWXWFMWWFWSFWMMWDMEWXFWFWFMVFMWVMSFGWFWQSSFSVVFFMVWSVVFWWWWWF
Iteration:
4135
Key:
['S', 'W', 'N', 'B', 'O', 'E', 'H', 'K', 'V', 'Y', 'D', 'A', 'T', 'C', 'J', 'G', 'L', 'U', 'Z', 'P', 'I', 'R', 'X', 'F', 'M', 'Q']
Deciphered:
XJLLAANJHAANEJHNALNNKAAEJANNKAEEEVNKEEEKJNNNNJEEANEAANAJAEENEKNHKANAKEMJNEANJANEHEEEAAKANHNNNAEKKENENNEKNHKANNAHNNHNANAHHKNAHJMNAQKNKJAAHNKKNKAENVNKANNKNEKNAANMAQNVKNKNKAHKANHAEKXNKNJEEKEHHKKAHNEHHKNNNNNK
Iteration:
4136
Key:
['Q', 'Z', 'J', 'L', 'A', 'H', 'N', 'M', 'I', 'O', 'G', 'U', 'P', 'X', 'F', 'T', 'V', 'E', 'R', 'W', 'C', 'D', 'S', 'Y', 'B', 'K']
Deciphered:
OXEEWWUXZWWUJXZUWEUUVWWJXWUUVWJJJSUVJJJVXUUUUXJJWUJWWUWXWJJUJVUZVWUWVJPXUJWUXWUJZJJJWWVWUZUUUWJVVJUJUUJVUZVWUUWZUU

Key:
['E', 'N', 'C', 'L', 'F', 'W', 'Y', 'S', 'H', 'T', 'R', 'G', 'O', 'B', 'V', 'Z', 'U', 'I', 'X', 'A', 'J', 'D', 'Q', 'M', 'K', 'P']
Deciphered:
EGTTHHCGYHHCMGYCHTCCVHHMGHCCVHMMMKCVMMMVGCCCCGMMHCMHHCHGHMMCMVCYVHCHVMJGCMHCGHCMYMMMHHVHCYCCCHMVVMCMCCMVCYVHCCHYCCYCHCHYYVCHYGJCHZVCVGHHYCVVCVHMCKCVHCCVCMVCHHCJHZCKVCVCVHYVHCYHMVECVCGMMVMYYVVHYCMYYVCCCCCV
Iteration:
4183
Key:
['H', 'N', 'L', 'X', 'K', 'W', 'T', 'Z', 'P', 'U', 'S', 'J', 'B', 'E', 'C', 'D', 'F', 'A', 'R', 'G', 'M', 'I', 'Q', 'O', 'V', 'Y']
Deciphered:
QZRRKKOZEKKOXZEOKROOPKKXZKOOPKXXXSOPXXXPZOOOOZXXKOXKKOKZKXXOXPOEPKOKPXGZOXKOZKOXEXXXKKPKOEOOOKXPPXOXOOXPOEPKOOKEOOEOKOKEEPOKEZGOKIPOPZKKEOPPOPKXOSOPKOOPOXPOKKOGKIOSPOPOPKEPKOEKXPQOPOZXXPXEEPPKEOXEEPOOOOOP
Iteration:
4184
Key:
['V', 'X', 'B', 'S', 'Q', 'G', 'O', 'M', 'P', 'L', 'J', 'F', 'N', 'K', 'Z', 'D', 'T', 'W', 'Y', 'H', 'E', 'I', 'R', 'U', 'A', 'C']
Deciphered:
LSYYDDZSNDDZGSNZDYZZPDDGSDZZPDGGGVZPGGGPSZZZZSGGDZGDDZDSDGGZGPZNPDZDPGQSZGDZSDZGNGGGDDPDZNZZZDGPPGZGZZGPZNPDZZDNZZ

Key:
['Q', 'V', 'S', 'T', 'G', 'J', 'A', 'N', 'D', 'L', 'X', 'H', 'Y', 'B', 'K', 'O', 'R', 'P', 'E', 'I', 'C', 'M', 'F', 'U', 'Z', 'W']
Deciphered:
WMGGCCUMOCCUPMOUCGUUICCPMCUUICPPPQUIPPPIMUUUUMPPCUPCCUCMCPPUPIUOICUCIPDMUPCUMCUPOPPPCCICUOUUUCPIIPUPUUPIUOICUUCOUUOUCUCOOIUCOMDUCRIUIMCCOUIIUICPUQUICUUIUPIUCCUDCRUQIUIUICOICUOCPIWUIUMPPIPOOIICOUPOOIUUUUUI
Iteration:
4230
Key:
['T', 'A', 'V', 'F', 'L', 'D', 'M', 'S', 'Q', 'U', 'X', 'B', 'H', 'P', 'G', 'W', 'E', 'Y', 'Z', 'R', 'O', 'N', 'I', 'J', 'C', 'K']
Deciphered:
DIBBHHYIZHHYUIZYHBYYFHHUIHYYFHUUUTYFUUUFIYYYYIUUHYUHHYHIHUUYUFYZFHYHFUAIYUHYIHYUZUUUHHFHYZYYYHUFFUYUYYUFYZFHYYHZYYZYHYHZZFYHZIAYHNFYFIHHZYFFYFHUYTYFHYYFYUFYHHYAHNYTFYFYFHZFHYZHUFDYFYIUUFUZZFFHZYUZZFYYYYYF
Iteration:
4231
Key:
['N', 'I', 'K', 'H', 'U', 'G', 'M', 'R', 'Q', 'O', 'T', 'W', 'F', 'A', 'B', 'P', 'C', 'V', 'S', 'D', 'X', 'L', 'J', 'Z', 'E', 'Y']
Deciphered:
MZNNSSQZCSSQJZCQSNQQTSSJZSQQTSJJJHQTJJJTZQQQQZJJSQJSSQSZSJJQJTQCTSQSTJKZQJSQZSQJCJJJSSTSQCQQQSJTTJQJQQJTQCTSQQSCQQ

KeyboardInterrupt: 